# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/common/schemas.ipynb"
    ]
}
```

# **TABLES**

---

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STATIC TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[CBS_INHABITANTS_PER_AGE_GROUP]'))
CREATE TABLE [VWSSTATIC].[CBS_INHABITANTS_PER_AGE_GROUP] (
	[ID] [INT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[GENDER] [VARCHAR](100),
	[AGE] [INT],
	[MARITAL_STATUS] [VARCHAR](100),
	[REGIONS] [VARCHAR](100),
	[PERIODS] [VARCHAR](100),
	[POPULATION_ON_FIRST_OF_JANUARY_1] [BIGINT],
	[AVERAGE_POPULATION_2] [BIGINT]
);
GO

-- 2) CREATE STATIC INDEX(ES).....
IF NOT EXISTS(SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCIX_DLI_CBS_INHABITANTS_PER_AGE_GROUP')
CREATE NONCLUSTERED INDEX [NCIX_DLI_CBS_INHABITANTS_PER_AGE_GROUP] ON [VWSSTATIC].[CBS_INHABITANTS_PER_AGE_GROUP] (
    [DATE_LAST_INSERTED]
) WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY];
GO

# **DATATINO CONFIGURATIONS**

---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'CBS_INHABITANTS_PER_AGE_GROUP',
        @workflow_alternative_name NVARCHAR(50),
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

SET @workflow_alternative_name = CONCAT(N'STATIC_', @workflow_name);

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_alternative_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_alternative_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_alternative_name,
    @description = @workflow_description,
    @schedule = '0 0 1 */3 *', -- AT 12:00 AM, ON DAY 1 OF THE MONTH, EVERY 3 MONTHS
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'https://opendata.cbs.nl/ODataApi/odata/03759ned/TypedDataSet?$filter=((Geslacht%20eq%20%27T001038%27))%20and%20((RegioS%20eq%20%27NL01%20%20%27))%20and%20((Perioden%20eq%20%272020JJ00%27))%20and%20((BurgerlijkeStaat%20eq%20%27T001019%27))'
    WHEN 'acceptance' THEN 'https://opendata.cbs.nl/ODataApi/odata/03759ned/TypedDataSet?$filter=((Geslacht%20eq%20%27T001038%27))%20and%20((RegioS%20eq%20%27NL01%20%20%27))%20and%20((Perioden%20eq%20%272020JJ00%27))%20and%20((BurgerlijkeStaat%20eq%20%27T001019%27))'
    ELSE 'https://opendata.cbs.nl/ODataApi/odata/03759ned/TypedDataSet?$filter=((Geslacht%20eq%20%27T001038%27))%20and%20((RegioS%20eq%20%27NL01%20%20%27))%20and%20((Perioden%20eq%20%272020JJ00%27))%20and%20((BurgerlijkeStaat%20eq%20%27T001019%27))'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'Web'
    WHEN 'acceptance' THEN 'Web'
    ELSE 'Web'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = CONCAT('SOURCE_', @workflow_name);
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = CONCAT('VWSSTATIC.', @workflow_name);

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'Geslacht|Leeftijd|BurgerlijkeStaat|RegioS|Perioden|BevolkingOp1Januari_1|GemiddeldeBevolking_2',
    @target_columns = 'GENDER|AGE|MARITAL_STATUS|REGIONS|PERIODS|POPULATION_ON_FIRST_OF_JANUARY_1|AVERAGE_POPULATION_2',
    @target_name = @target_name,
    @source_type = 'JsonFile',
    @location_type = @location_type,
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = CONCAT('PROCESS_', @workflow_name);
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_alternative_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_alternative_name,
    @active = 1;
GO